In [36]:
from inspect import trace
import pandas as pd
from pandas import json_normalize
import requests
import urllib3
import os
from sqlalchemy import create_engine
import sqlalchemy
import psycopg2
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
from time import sleep
from datetime import datetime
warnings.filterwarnings("ignore")

In [37]:
def get_access_token(client_id, client_secret, refresh_token):
    payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
    }
    auth_url = "https://www.strava.com/oauth/token"
    print("Requesting Token...\n")
    res = requests.post(auth_url, data=payload, verify=False)
    access_token = res.json()['access_token']
    print("Access Token = {}\n".format(access_token))
    return access_token

In [38]:
def get_base_activities(access_token, start_date):
    # Convert date to epoch
    utc_time = datetime.strptime(start_date, "%Y-%m-%d")
    epoch_time = (utc_time - datetime(1970, 1, 1)).total_seconds()
    activities_url = 'https://www.strava.com/api/v3/activities'
    header = {'Authorization': 'Bearer ' + access_token}
    for i in range(100):
        page = i+1
        param = {'per_page': 200, 'after': epoch_time, 'page':page}
        print(f"Collecting page {page}")
        my_dataset = requests.get(activities_url, headers=header, params=param).json()
        if i == 0:
            activities = json_normalize(my_dataset)
        else:
            activities = activities.append(json_normalize(my_dataset))
        if len(json_normalize(my_dataset)) == 0:
            break
    return activities

In [39]:
def get_activity_detail(access_token, activity_id):
    cols_to_keep = ['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id', 'start_date', 'start_date_local', 'timezone', 'utc_offset', 'location_city', 'location_state', 'location_country', 'achievement_count', 'kudos_count', 'comment_count', 'athlete_count', 'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed', 'average_temp', 'average_watts', 'kilojoules', 'device_watts', 'has_heartrate', 'average_heartrate', 'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed', 'suffer_score', 'description', 'calories', 'perceived_exertion', 'prefer_perceived_exertion', 'hide_from_home', 'device_name', 'embed_token', 'available_zones', 'athlete.id', 'athlete.resource_state', 'map.id', 'map.polyline', 'map.resource_state', 'map.summary_polyline', 'gear.id', 'gear.primary', 'gear.name', 'gear.nickname', 'gear.resource_state', 'gear.retired', 'gear.distance', 'gear.converted_distance', 'photos.primary', 'photos.count']
    activities_url = 'https://www.strava.com/api/v3/activities'
    activities_detail_url = activities_url + '/' + str(activity_id) + '?access_token=' + access_token
    header = {'Authorization': 'Bearer ' + access_token}
    activity_detail = json_normalize(requests.get(activities_detail_url, headers=header).json())
    segments = json_normalize(activity_detail['segment_efforts'][0])
    activity_detail = activity_detail[activity_detail.columns.intersection(cols_to_keep)]
    return segments, activity_detail

In [40]:
def to_seconds(timestr):

    timestr = str(timestr).replace("s", "")
    seconds= 0
    for part in timestr.split(':'):
        try: 
            seconds= seconds*60 + int(part, 10)
        except:
            seconds = 0
    return seconds

In [41]:
def clean_segments(segments):
    segments['segment.start_latitude'] = segments['segment.start_latlng'].str[0]; segments['segment.start_longitude'] = segments['segment.start_latlng'].str[1]
    segments['segment.end_latitude'] = segments['segment.end_latlng'].str[0]; segments['segment.end_longitude'] = segments['segment.end_latlng'].str[1]
    return segments

In [42]:
def clean_segment_detail(segment_detail):
    segment_detail['xoms.overall'] = segment_detail['xoms.overall'].apply(to_seconds)
    return segment_detail

In [43]:
def get_segment_efforts(segment_id, access_token):
    segment_url = 'https://www.strava.com/api/v3/segments/'
    url = segment_url + str(segment_id)
    header = {'Authorization': 'Bearer ' + access_token}
    segment_json = requests.get(url, headers=header).json()
    segment_temp = json_normalize(segment_json)
    return segment_temp

In [58]:
def load(df, tbl):
    postgresuser = 'postgres'
    postgrespass = 'Crosby87'
    server = 'localhost'
    try:
        rows_imported = 0
        engine = create_engine(f'postgresql://{postgresuser}:{postgrespass}@{server}:5432/strava')  
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        df.to_sql(f'stg_{tbl}', engine, if_exists='replace', index=False, dtype={"achievements": sqlalchemy.types.JSON})
    finally:
        exit

In [61]:
def run_strava_etl(start_date):
	segment_url = 'https://www.strava.com/api/v3/segments/'
	
	# Define parameters to get the access token
	client_id = "42399"; client_secret = '750191abe2d99529b503cbfe70b86ad8f3d8ae70'; refresh_token = '4053c255fe9432ade612c43f4bf83f59f3153316'
	
	access_token = get_access_token(client_id, client_secret, refresh_token)
	
	#Collect base activities
	base_activities = get_base_activities(access_token, start_date)
	
	#Filter base activities to only non-trainer
	activities = base_activities[base_activities['trainer'] == False]
	num_activities = len(activities)
	print(f"Collected {num_activities} base activities that aren't on the trainer...")
	print(activities.tail)
	
	#Loop through activities to get the activity details and segments completed
	print("Collecting activity detail and segments...")
	for j in range(num_activities):
		segment, activity_detail = get_activity_detail(access_token, activities.iloc[j]['id'])
		if j == 0:
			segments = segment
			activity_details = activity_detail
		else:    
			segments = segments.append(segment, ignore_index=True)  
			activity_details = activity_details.append(activity_detail, ignore_index=True)
		if (j > 0) & (j % 550 == 0):
			print(f'Sleeping at {j} of {num_activities} for rate limiting reasons...')
			sleep(900)
			print('Back to work!')
	
	print("Done")
	
	#Clean the segment data
	print("Cleaning data...")
	segments = clean_segments(segments)
	
	# Get a distinct list of segments 
	distinct_segment_ids = segments['segment.id'].unique()
	num_segment_ids = len(distinct_segment_ids)
	cnt = j
	cnt_new = 0
	
	print(f"Collecting {num_segment_ids} segments efforts...")
	
	for id in distinct_segment_ids:
		segment_temp = get_segment_efforts(id, access_token)
		if cnt == j:
			segment_detail = segment_temp
		else:
			segment_detail = segment_detail.append(segment_temp)
		cnt += 1
		cnt_new += 1
		total_seg = len(distinct_segment_ids)
		if cnt % 550 == 0:
			print(f'Sleeping at {cnt_new} of {total_seg} for rate limiting reasons...')
			sleep(900)
			print('Back to work!')
			
	segment_detail = clean_segment_detail(segment_detail)
	
	load(segments, 'segment_efforts')
	load(activity_details, 'activity_detail')
	load(segment_detail, 'segments')
	
	print("Complete!")

In [62]:
sleep(900)
run_strava_etl('2018-12-12')

Requesting Token...

Access Token = eb6d1b46c5201be6c1ec13244fa9c028016700e9

Collected 946 base activities that aren't on the trainer...
<bound method NDFrame.tail of      resource_state                                               name  \
0                 2  Arlington - Prince of Peace Roman Catholic School   
1                 2                          Providence Ridge - Arcola   
2                 2                              Arlington - Allentown   
3                 2                 Arlington - South Side High School   
4                 2                            Arlington - Rex Theatre   
..              ...                                                ...   
98                2                                         Lunch Ride   
102               2                                       Morning Ride   
103               2                                     Afternoon Ride   
107               2                                     Afternoon Ride   
108               